In [1]:
import re
from time import time
import collections
# from drug_entity_chunker_v2 import ScikitLearnChunker
from word2features import *
from nltk import download
# from nltk.stem.snowball import SnowballStemmer
# from nltk import word_tokenize
from parser_utils_v2 import  parse_drug_iob_xy
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction import DictVectorizer
import numpy as np
# from nltk import tree2conlltags
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score
# from sklearn.externals import joblib

# from itertools import chain
import nltk
import sklearn
import sys
sys.path.append("C:/Users/ALEIX/Documents/GitHub/MAI_AHLT_DDIproject/python-crfsuite-master")
import pycrfsuite
!pip install python-crfsuite
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# NLTK dependencies
download('punkt')
download('averaged_perceptron_tagger')

# Directories to find MedLine and DrugBank train and test data
DRUGBANK_TRAIN_DIR = './Train/DrugBank'
DRUGBANK_TEST_DIR = './Test/Test for DrugNER task/DrugBank'
MEDLINE_TRAIN_DIR = './Train/MedLine'
MEDLINE_TEST_DIR = './Test/Test for DrugNER task/MedLine'

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ALEIX\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ALEIX\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
tags = ['drug', 'brand', 'group', 'drug_n']

print("Loading DRUGBANK train data...")
t0 = time()

drugb_x_train_dic, drugb_y_train_dic, _, _ = parse_drug_iob_xy(DRUGBANK_TRAIN_DIR, dict_features, tags=tags)
drugb_x_train_arr, drugb_y_train_arr, _, _ = parse_drug_iob_xy(DRUGBANK_TRAIN_DIR, array_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("Loading DRUGBANK test data...")
t0 = time()
drugb_x_test_dic, drugb_y_test_dic, _, _ = parse_drug_iob_xy(DRUGBANK_TEST_DIR, dict_features, tags=tags)
drugb_x_test_arr, drugb_y_test_arr, drugb_sId_test, drugb_words_start = parse_drug_iob_xy(DRUGBANK_TEST_DIR, array_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("train instances: ", len(drugb_x_train_dic), "test instances: ", len(drugb_x_test_dic))

print("\n Loading MEDLINE train data...")
t0 = time()

med_x_train_dic, med_y_train_dic, _, _ = parse_drug_iob_xy(MEDLINE_TRAIN_DIR, dict_features, tags=tags)
med_x_train_arr, med_y_train_arr, _, _ = parse_drug_iob_xy(MEDLINE_TRAIN_DIR, array_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("Loading MEDLINE test data...")
t0 = time()
med_x_test_dic, med_y_test_dic, _, _ = parse_drug_iob_xy(MEDLINE_TEST_DIR, dict_features, tags=tags)
med_x_test_arr, med_y_test_arr, med_sId_test, med_words_start = parse_drug_iob_xy(MEDLINE_TEST_DIR, array_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("train instances: ", len(med_x_train_dic), "test instances: ", len(med_x_test_dic))

Loading DRUGBANK train data...
done in 148.348941s 

Loading DRUGBANK test data...
done in 3.975694s 

train instances:  119978 test instances:  2929

 Loading MEDLINE train data...
done in 36.055856s 

Loading MEDLINE test data...
done in 14.718377s 

train instances:  31187 test instances:  12607


In [8]:
tags = ['drug', 'brand', 'group', 'drug_n']

print("DATA SKLEARN MODELS")
print("Loading DRUGBANK train data...")
t0 = time()
drugb_x_train_dic, drugb_y_train_dic, _, _ = parse_drug_iob_xy(DRUGBANK_TRAIN_DIR, dict_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("Loading DRUGBANK test data...")
t0 = time()
drugb_x_test_dic, drugb_y_test_dic, _, _ = parse_drug_iob_xy(DRUGBANK_TEST_DIR, dict_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("train instances: ", len(drugb_x_train_dic), "test instances: ", len(drugb_x_test_dic))

print("\n Loading MEDLINE train data...")
t0 = time()
med_x_train_dic, med_y_train_dic, _, _ = parse_drug_iob_xy(MEDLINE_TRAIN_DIR, dict_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("Loading MEDLINE test data...")
t0 = time()
med_x_test_dic, med_y_test_dic, _, _ = parse_drug_iob_xy(MEDLINE_TEST_DIR, dict_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("train instances: ", len(med_x_train_dic), "test instances: ", len(med_x_test_dic))

DATA SKLEARN MODELS
Loading DRUGBANK train data...


KeyboardInterrupt: 

In [9]:
classes = ['O', 'B-drug', 'I-drug', 'B-brand', 'I-brand', 'B-group', 'I-group', 'B-drug_n', 'I-drug_n']

print("Adapting DRUGBANK data to numeric format")
t0 = time()
vectorizer = DictVectorizer(sparse=True)
drugb_x_train_dic_vec = vectorizer.fit_transform(drugb_x_train_dic)
drugb_x_test_dic_vec = vectorizer.transform(drugb_x_test_dic)

drugb_y_train_dic_vec = vectorize_attr(drugb_y_train_dic, classes)   
drugb_y_test_dic_vec = vectorize_attr(drugb_y_test_dic, classes)

drugb_y_test_arr_vec = vectorize_attr(drugb_y_test_arr, classes)

duration = time() - t0
print("done in %fs \n" % (duration))

print("Adapting MEDLINE data to numeric format")
t0 = time()
med_x_train_dic_vec = vectorizer.transform(med_x_train_dic)
med_x_test_dic_vec = vectorizer.transform(med_x_test_dic)

med_y_train_dic_vec = vectorize_attr(med_y_train_dic, classes)   
med_y_test_dic_vec = vectorize_attr(med_y_test_dic, classes)

med_y_test_arr_vec = vectorize_attr(med_y_test_arr, classes)

duration = time() - t0; print("done in %fs \n" % (duration))

print("Unifying both datasets data")
datasetNames = ['DrugBank', 'MedLine']
x_train_dic_vec = [drugb_x_train_dic_vec, med_x_train_dic_vec]
y_train_dic_vec = [drugb_y_train_dic_vec, med_y_train_dic_vec]

x_test_dic_vec = [drugb_x_test_dic_vec, med_x_test_dic_vec]
y_test_dic_vec = [drugb_y_test_dic_vec, med_y_test_dic_vec]

y_test_arr_vec = [drugb_y_test_arr_vec, med_y_test_arr_vec]

x_train_arr = [drugb_x_train_arr, med_x_train_arr]
y_train_arr = [drugb_y_train_arr, med_y_train_arr]

x_test_arr = [drugb_x_test_arr, med_x_test_arr]
y_test_arr = [drugb_y_test_arr, med_y_test_arr]

duration = time() - t0; print("done in %fs" % (duration))

Adapting DRUGBANK data to numeric format
done in 4.112657s 

Adapting MEDLINE data to numeric format
done in 1.202290s 

Unifying both datasets data
done in 1.203290s


In [3]:
classes = ['O', 'B-drug', 'I-drug', 'B-brand', 'I-brand', 'B-group', 'I-group', 'B-drug_n', 'I-drug_n']

print("Adapting DRUGBANK data to numeric format"); t0 = time()
vectorizer = DictVectorizer(sparse=True)
drugb_x_train_dic_vec = vectorizer.fit_transform(drugb_x_train_dic)
drugb_x_test_dic_vec = vectorizer.transform(drugb_x_test_dic)

drugb_y_train_dic_vec = vectorize_attr(drugb_y_train_dic, classes)   
drugb_y_test_dic_vec = vectorize_attr(drugb_y_test_dic, classes)

duration = time() - t0; print("done in %fs \n" % (duration))

print("Adapting MEDLINE data to numeric format"); t0 = time()
med_x_train_dic_vec = vectorizer.transform(med_x_train_dic)
med_x_test_dic_vec = vectorizer.transform(med_x_test_dic)

med_y_train_dic_vec = vectorize_attr(med_y_train_dic, classes)   
med_y_test_dic_vec = vectorize_attr(med_y_test_dic, classes)
duration = time() - t0; print("done in %fs \n" % (duration))

print("Unifying both datasets data")
datasetNames = ['DrugBank', 'MedLine']
x_train_dic_vec = [drugb_x_train_dic_vec, med_x_train_dic_vec]
y_train_dic_vec = [drugb_y_train_dic_vec, med_y_train_dic_vec]

x_test_dic_vec = [drugb_x_test_dic_vec, med_x_test_dic_vec]
y_test_dic_vec = [drugb_y_test_dic_vec, med_y_test_dic_vec]
duration = time() - t0; print("done in %fs" % (duration))

Adapting DRUGBANK data to numeric format


NameError: name 'drugb_y_test_arr' is not defined

In [ ]:
# Define classifier models
clf_perceptron5 = Perceptron(verbose=0, n_jobs=-1, max_iter=5)
clf_perceptron50 = Perceptron(verbose=0, n_jobs=-1, max_iter=50)
clf_perceptron100 = Perceptron(verbose=0, n_jobs=-1, max_iter=100)
clf_tree = DecisionTreeClassifier(criterion='entropy')
clf_NB = MultinomialNB()
clf_MLP = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100), random_state=1)

#  (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
#         (Perceptron(n_iter=50), "Perceptron"),
#         (PassiveAggressiveClassifier(n_iter=50), "Passive-Aggressive"),
#         (KNeighborsClassifier(n_neighbors=10), "kNN"),
#         (RandomForestClassifier(n_estimators=100), "Random forest")):

# for penalty in ["l2", "l1"]:
#     print('=' * 80)
#     print("%s penalty" % penalty.upper())
#     # Train Liblinear model
#     results.append(benchmark(LinearSVC(penalty=penalty, dual=False,
#                                        tol=1e-3)))

#     # Train SGD model
#     results.append(benchmark(SGDClassifier(alpha=.0001, n_iter=50,
#                                            penalty=penalty)))
# LinearSVC(penalty="l2"))
# http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html

names = ["Perceptron5 ", "Perceptron 50", "Perceptron 100","Decision tree", "Naive Bayes", "Multi Layer Perceptron"]
clfs = [clf_perceptron5, clf_perceptron50, clf_perceptron100, clf_tree, clf_NB, clf_MLP]

print("Sklearn models defined")

# Training of sklearn models

In [266]:
import warnings
warnings.filterwarnings('ignore')
f1_scores = []
print("Training classifiers")
for clfName, clf in zip(names, clfs):
    inner_f1_scores = []
    for x_train, y_train, x_test, y_test, datasetName in zip(x_train_dic_vec, y_train_dic_vec, x_test_dic_vec, y_test_dic_vec, datasetNames):
        printmd("**{} with {} dataset**".format(clfName, datasetName))
        print("Training...")
        clf.fit(x_train, y_train)

        print("Testing...")
        y_pred = clf.predict(x_test)
        scores = cross_val_score(clf, x_test, y_test, cv=5)
        inner_f1_scores.append(f1_score(y_test, y_pred, average='micro')) 

        print("Classification report")
        print(classification_report(y_test, y_pred, target_names=classes))
        print("\n CV scores: ", scores, ", mean: ", np.mean(scores),"\n")

        print("Confusion matrix")
        print(confusion_matrix(y_test, y_pred))
        print("\n")
    f1_scores.append(inner_f1_scores)

Training classifiers


**Perceptron5  with DrugBank dataset**

Training...
Testing...
Classification report
             precision    recall  f1-score   support

          O       0.98      0.98      0.98      2569
     B-drug       0.87      0.79      0.83       179
     I-drug       0.76      0.65      0.70        20
    B-brand       0.71      0.74      0.72        53
    I-brand       0.33      1.00      0.50         1
    B-group       0.83      0.83      0.83        64
    I-group       0.83      0.94      0.88        31
   B-drug_n       1.00      0.17      0.29         6
   I-drug_n       1.00      0.83      0.91         6

avg / total       0.96      0.96      0.96      2929


 CV scores:  [ 0.9220339   0.93515358  0.89931741  0.92478632  0.90893471] , mean:  0.918045184151 

Confusion matrix
[[2530   13    2   13    1    5    5    0    0]
 [  27  141    2    3    1    5    0    0    0]
 [   6    1   13    0    0    0    0    0    0]
 [   8    5    0   39    0    1    0    0    0]
 [   0    0    0    0    1    0    0    0    0]
 [  10    

**Perceptron5  with MedLine dataset**

Training...
Testing...
Classification report
             precision    recall  f1-score   support

          O       0.98      0.99      0.98     12033
     B-drug       0.60      0.33      0.43       168
     I-drug       0.86      0.64      0.73        28
    B-brand       0.00      0.00      0.00         6
    I-brand       0.00      0.00      0.00         0
    B-group       0.73      0.36      0.48        90
    I-group       0.68      0.93      0.78        58
   B-drug_n       0.25      0.18      0.21       114
   I-drug_n       0.88      0.84      0.86       110

avg / total       0.96      0.97      0.96     12607


 CV scores:  [ 0.96315372  0.96987713  0.96195006  0.9511711   0.96544877] , mean:  0.962320156521 

Confusion matrix
[[11917    23     3     0     3     9    22    45    11]
 [   99    56     0     0     0     1     0    12     0]
 [    8     0    18     0     0     0     0     0     2]
 [    4     1     0     0     0     1     0     0     0]
 [    0     0     0   

...

Training...
Testing...
Classification report
             precision    recall  f1-score   support

          O       0.96      1.00      0.98     12033
     B-drug       0.88      0.09      0.16       168
     I-drug       0.00      0.00      0.00        28
    B-brand       0.00      0.00      0.00         6
    I-brand       0.00      0.00      0.00        90
    B-group       0.00      0.00      0.00        58
    I-group       0.00      0.00      0.00       114
   B-drug_n       0.00      0.00      0.00       110

avg / total       0.92      0.96      0.93     12607


 CV scores:  [ 0.95364501  0.95402299  0.95402299  0.95514093  0.95552025] , mean:  0.954470433609 

Confusion matrix
[[12031     2     0     0     0     0     0     0]
 [  153    15     0     0     0     0     0     0]
 [   28     0     0     0     0     0     0     0]
 [    6     0     0     0     0     0     0     0]
 [   90     0     0     0     0     0     0     0]
 [   58     0     0     0     0     0     0     

**Multi Layer Perceptron with DrugBank dataset**

Training...
Testing...
Classification report
             precision    recall  f1-score   support

          O       0.98      0.99      0.99      2569
     B-drug       0.92      0.80      0.86       179
     I-drug       0.81      0.65      0.72        20
    B-brand       0.77      0.77      0.77        53
    I-brand       0.33      1.00      0.50         1
    B-group       0.78      0.89      0.83        64
    I-group       0.86      1.00      0.93        31
   B-drug_n       0.50      0.17      0.25         6
   I-drug_n       1.00      1.00      1.00         6

avg / total       0.97      0.97      0.97      2929


 CV scores:  [ 0.91355932  0.93515358  0.90273038  0.93675214  0.91408935] , mean:  0.920456952982 

Confusion matrix
[[2539    5    1    7    2   11    4    0    0]
 [  24  143    1    5    0    4    1    1    0]
 [   6    1   13    0    0    0    0    0    0]
 [   8    4    0   41    0    0    0    0    0]
 [   0    0    0    0    1    0    0    0    0]
 [   6    

**Multi Layer Perceptron with MedLine dataset**

Training...
Testing...
Classification report
             precision    recall  f1-score   support

          O       0.98      0.98      0.98     12033
     B-drug       0.43      0.51      0.47       168
     I-drug       0.82      0.64      0.72        28
    B-brand       0.00      0.00      0.00         6
    I-brand       0.57      0.39      0.46        90
    B-group       0.83      0.83      0.83        58
    I-group       0.24      0.25      0.25       114
   B-drug_n       0.88      0.74      0.80       110

avg / total       0.96      0.96      0.96     12607


 CV scores:  [ 0.96077655  0.96987713  0.96313912  0.96069869  0.9618745 ] , mean:  0.963273197838 

Confusion matrix
[[11842    74     4     0    22    10    72     9]
 [   65    85     0     0     2     0    16     0]
 [    8     0    18     0     0     0     0     2]
 [    3     2     0     0     1     0     0     0]
 [   40    12     0     0    35     0     3     0]
 [   10     0     0     0     0    48     0     

In [14]:
# free memory deleting unneccessary variables
import sys
# who
for v in dir():
    if not v.startswith('_') and 'dic' in v:
        del globals()[v]
#dir()

# Training of other models

In [3]:
tags = ['drug', 'brand', 'group', 'drug_n']

print("DATA FOR CRF MODEL")
print("Loading DRUGBANK train data...")
t0 = time()
drugb_x_train_arr, drugb_y_train_arr, _, _ = parse_drug_iob_xy(DRUGBANK_TRAIN_DIR, array_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("Loading DRUGBANK test data...")
t0 = time()
drugb_x_test_arr, drugb_y_test_arr, drugb_sId_test, drugb_words_start = parse_drug_iob_xy(DRUGBANK_TEST_DIR, array_features, tags=tags)
duration = time() - t0
print("done in %fs \n" % (duration))

print("train instances: ", len(drugb_x_train_arr), "test instances: ", len(drugb_x_test_arr))

# print("\n Loading MEDLINE train data...")
# t0 = time()
# med_x_train_arr, med_y_train_arr, _, _ = parse_drug_iob_xy(MEDLINE_TRAIN_DIR, array_features, tags=tags)
# duration = time() - t0
# print("done in %fs \n" % (duration))

# print("Loading MEDLINE test data...")
# t0 = time()
# med_x_test_arr, med_y_test_arr, med_sId_test, med_words_start = parse_drug_iob_xy(MEDLINE_TEST_DIR, array_features, tags=tags)
# duration = time() - t0
# print("done in %fs \n" % (duration))

# print("train instances: ", len(med_x_train_arr), "test instances: ", len(med_x_test_arr))

DATA FOR CRF MODEL
Loading DRUGBANK train data...
done in 71.798421s 

Loading DRUGBANK test data...
done in 1.887570s 

train instances:  119978 test instances:  2929


In [4]:
classes = ['O', 'B-drug', 'I-drug', 'B-brand', 'I-brand', 'B-group', 'I-group', 'B-drug_n', 'I-drug_n']

print("Adapting DRUGBANK data to numeric format"); t0 = time()
drugb_y_test_arr_vec = vectorize_attr(drugb_y_test_arr, classes)
duration = time() - t0; print("done in %fs \n" % (duration))

# print("Adapting MEDLINE data to numeric format"); t0 = time()
# med_y_test_arr_vec = vectorize_attr(med_y_test_arr, classes)
# duration = time() - t0; print("done in %fs \n" % (duration))

# print("Unifying both datasets data")
# datasetNames = ['DrugBank', 'MedLine']
# y_test_arr_vec = [drugb_y_test_arr_vec, med_y_test_arr_vec]

# x_train_arr = [drugb_x_train_arr, med_x_train_arr]
# y_train_arr = [drugb_y_train_arr, med_y_train_arr]

# x_test_arr = [drugb_x_test_arr, med_x_test_arr]
# y_test_arr = [drugb_y_test_arr, med_y_test_arr]

duration = time() - t0; print("done in %fs" % (duration))

Adapting DRUGBANK data to numeric format
done in 0.004995s 

done in 0.005996s


In [11]:
from sklearn.metrics import f1_score, classification_report
from collections import Counter

printmd("**CONDITIONAL RANDOM FIELDS MODEL**")


inner_f1_scores = []
for x_train, y_train, x_test, y_test, y_test_vec, datasetName in zip(x_train_arr, y_train_arr, x_test_arr, y_test_arr, y_test_arr_vec, datasetNames):   
    printmd("**{} with {} dataset**".format('CRF', datasetName))    
    print("Preparing training CRF file"); t0 = time()  
    trainer = pycrfsuite.Trainer(verbose=False)
    for xseq, yseq in zip(x_train, y_train):
        trainer.append([xseq], [yseq])

    duration = time() - t0; print("done in %fs \n" % (duration))

    trainer.set_params({
        'c1': 1.0,   # coefficient for L1 penalty
        'c2': 1e-3,  # coefficient for L2 penalty
        'max_iterations': 50,  # stop earlier
        # include transitions that are possible, but not observed
        'feature.possible_transitions': True
    })
    trainer.params()

    print("Training CRF"); t0 = time()
    trainer.train('conll2002-esp.crfsuite')
    duration = time() - t0; print("done in %fs \n" % (duration))

    print("Preparing CRF tagger"); duration = time() - t0
    tagger = pycrfsuite.Tagger(); tagger.open('conll2002-esp.crfsuite')
    duration = time() - t0; print("done in %fs" % (duration))

    print("Testing"); t0 = time()
    y_pred_out = [tagger.tag([xseq]) for xseq in x_test]
    y_pred = []
    [y_pred.append(pred[0]) for pred in y_pred_out]
    y_pred_vec = vectorize_attr(y_pred, classes)

    duration = time() - t0; print("done in %fs" % (duration))
    
    f1_score = f1_score(y_test_vec, y_pred_vec, average='micro')
    inner_f1_scores.append(f1_score) 

    print("F1 score: ", f1_score,"\n")
    print("Classification report"); print(classification_report(y_test_vec, y_pred_vec, target_names=classes))

    print("Confusion matrix"); print(confusion_matrix(y_test_vec, y_pred_vec))
    
    info = tagger.info()
    print("Top positive:"); print_state_features(Counter(info.state_features).most_common(20))
    print("\nTop negative:"); print_state_features(Counter(info.state_features).most_common()[-20:])
    
f1_scores.append(inner_f1_scores)

**CONDITIONAL RANDOM FIELDS MODEL**

**CRF with DrugBank dataset**

Preparing training CRF file
done in 2.652581s 

Training CRF
done in 9.775827s 

Preparing CRF tagger
done in 9.776824s
Testing
done in 0.070959s
y_test_vec:  2929 

y_pred_vec:  2929 

F1 score:  0.968931375896 

Classification report
             precision    recall  f1-score   support

          O       0.98      0.99      0.99      2569
     B-drug       0.95      0.79      0.86       179
     I-drug       0.84      0.80      0.82        20
    B-brand       0.77      0.77      0.77        53
    I-brand       0.50      1.00      0.67         1
    B-group       0.88      0.81      0.85        64
    I-group       0.91      1.00      0.95        31
   B-drug_n       0.50      0.17      0.25         6
   I-drug_n       1.00      1.00      1.00         6

avg / total       0.97      0.97      0.97      2929

Confusion matrix
[[2549    4    1    6    1    5    3    0    0]
 [  28  141    2    5    0    2    0    1    0]
 [   4    0   16    0    0    0    0    0    0]
 [  10    2    0 

**CRF with MedLine dataset**

Preparing training CRF file
done in 0.814546s 

Training CRF
done in 2.569054s 

Preparing CRF tagger
done in 2.570055s
Testing
done in 0.216948s
y_test_vec:  12607 

y_pred_vec:  12607 



TypeError: 'numpy.float64' object is not callable

In [14]:
from sklearn.metrics import f1_score, classification_report
from collections import Counter

printmd("**CONDITIONAL RANDOM FIELDS MODEL WITH DRUGBANK**")

inner_f1_scores = []

printmd("**{} with {} dataset**".format('CRF','DRUGBANK'))    
print("Preparing training CRF file"); t0 = time()  
trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(med_x_train_arr, med_y_train_arr):
    trainer.append([xseq], [yseq])

duration = time() - t0; print("done in %fs \n" % (duration))

trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier
    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})
trainer.params()

print("Training CRF"); t0 = time()
trainer.train('conll2002-esp.crfsuite')
duration = time() - t0; print("done in %fs \n" % (duration))

print("Preparing CRF tagger"); duration = time() - t0
tagger = pycrfsuite.Tagger(); tagger.open('conll2002-esp.crfsuite')
duration = time() - t0; print("done in %fs" % (duration))

print("Testing"); t0 = time()
print(len(med_x_test_arr))
y_pred_outer = [tagger.tag([xseq]) for xseq in med_x_test_arr]
print(len(y_pred_outer))
[y_pred.append(pred[0]) for pred in y_pred_outer]
print(len(y_pred_vec,'aqui'))
y_pred_vec = vectorize_attr(y_pred, classes)
print(len(y_pred_vec))
duration = time() - t0; print("done in %fs" % (duration))

f1_score = f1_score(med_y_test_arr_vec, y_pred_vec, average='micro')
inner_f1_scores.append(f1_score) 

print("F1 score: ", f1_score,"\n")
print("Classification report"); print(classification_report(med_y_test_arr_vec, y_pred_vec, target_names=classes))

print("Confusion matrix"); print(confusion_matrix(med_y_test_arr_vec, y_pred_vec))

info = tagger.info()
print("Top positive:"); print_state_features(Counter(info.state_features).most_common(20))
print("\nTop negative:"); print_state_features(Counter(info.state_features).most_common()[-20:])
    
# f1_scores.append(inner_f1_scores)

**CONDITIONAL RANDOM FIELDS MODEL WITH DRUGBANK**

**CRF with DRUGBANK dataset**

Preparing training CRF file
done in 0.751566s 

Training CRF
done in 2.561678s 

Preparing CRF tagger
done in 2.563678s
Testing
12607
12607
50428
done in 2.062881s


ValueError: Found input variables with inconsistent numbers of samples: [12607, 50428]

In [ ]:
med_x_test_arr

### Some observations
#### 1. Almost all top positive cases are B-drug entities, what is evident taking into account that the most entities are drugs and they are not formed by compound nouns.
#### 2. 

In [228]:


bestIdx = [i for i, j in enumerate(f1_scores) if j == max(f1_scores)][0]
if bestIdx < len(names):
    print("The best classifier is ", names[bestIdx], " one of the sklearn classifiers list")
else:
    print("The best classifier is CRF, NOT in sklearn classifiers list")

The best classifier is CRF, NOT in sklearn classifiers list


## ONLY IF best model is in sklearn classifiers list, execute the following block

In [238]:
clf = clfs[bestIdx]
print("Training...")
t0 = time()
clf.fit(x_train_vec, y_train_vec)
duration = time() - t0
print("done in %fs \n" % (duration))

print("Writing output TEST file")
t0 = time()
words_chain = []
with open("semEval_task1_output.txt", 'w') as outfile:
    for sId, w_start, x in zip(sId_test, words_start, x_test_dic_vec):
        
        prediction = clf.predict(x)
        prediction = classes[int(prediction[0])]
        
        if 'B' in prediction or 'I' in prediction:
            
            entity = prediction.replace('B-', '').replace('I-', '')
            word_data = vectorizer.inverse_transform(x)[0]
            
            for key in word_data:
                if 'word' in str(key) and not('next' in str(key)) and not('prev' in str(key)):
                    word = str(key).replace('word=', ''); break
            
            if 'B' in prediction:
                if len(words_chain) > 0:
                    outfile.write("{}|{}|{}|{}\n".format(words_chain[0], words_chain[1], words_chain[2], words_chain[3]))
                    words_chain = []
                  
                words_chain.append(sId)
                words_chain.append('{}-{}'.format(w_start, w_start + len(word) - 1))
                words_chain.append(word)
                words_chain.append(entity)
                
            else:
                if len(words_chain) > 0:
                    if words_chain[0] == sId: # if entity has more than 1 word                
                        words_chain[1] = '{};{}-{}'.format(words_chain[1], w_start,w_start + len(word))
                        words_chain[2] = '{} {}'.format(words_chain[2], word)
                        
                    else:  # if by mistake a 'I-{*}' arises and we had accumulated 'B-{*}' or 'I-{*}' from last sentence
                        outfile.write("{}|{}|{}|{}\n".format(words_chain[0], words_chain[1], words_chain[2], words_chain[3]))
                        words_chain = []
                        
                        words_chain.append(sId)
                        words_chain.append('{}-{}'.format(w_start, w_start + len(word) - 1))
                        words_chain.append(word)
                        words_chain.append(entity)
                      
                else:  # if by mistake a 'I-{*}' arises but there is no previous 'B-{*}'

                    words_chain.append(sId)
                    words_chain.append('{}-{}'.format(w_start, w_start + len(word) - 1))
                    words_chain.append(word)
                    words_chain.append(entity)
                    
        else:
            if len(words_chain) > 0:
                outfile.write("{}|{}|{}|{}\n".format(words_chain[0], words_chain[1], words_chain[2], words_chain[3]))
              
            words_chain = []
      
duration = time() - t0
print("done in %fs" % (duration))

IndexError: list index out of range

## ONLY IF best model is CRF, execute the following block

In [239]:
print("Writing output TEST file")
t0 = time()
words_chain = []
with open("semEval_task1_output.txt", 'w') as outfile:
    for sId, w_start, x in zip(sId_test, words_start, x_test_arr):
        
        prediction = tagger.tag([x])[0]
        
        if 'B' in prediction or 'I' in prediction:
            
            entity = prediction.replace('B-', '').replace('I-', '')
            
            for element in x:
                if 'word' in element and not('next' in element) and not('prev' in element):
                    word = element.replace('word=', ''); break
            
            if 'B' in prediction:
                if len(words_chain) > 0:
                    outfile.write("{}|{}|{}|{}\n".format(words_chain[0], words_chain[1], words_chain[2], words_chain[3]))
                    words_chain = []
                  
                words_chain.append(sId)
                words_chain.append('{}-{}'.format(w_start, w_start + len(word) - 1))
                words_chain.append(word)
                words_chain.append(entity)
                
            else:
                if len(words_chain) > 0:
                    if words_chain[0] == sId: # if entity has more than 1 word                
                        words_chain[1] = '{};{}-{}'.format(words_chain[1], w_start,w_start + len(word))
                        words_chain[2] = '{} {}'.format(words_chain[2], word)
                        
                    else:  # if by mistake a 'I-{*}' arises and we had accumulated 'B-{*}' or 'I-{*}' from last sentence
                        outfile.write("{}|{}|{}|{}\n".format(words_chain[0], words_chain[1], words_chain[2], words_chain[3]))
                        words_chain = []
                        
                        words_chain.append(sId)
                        words_chain.append('{}-{}'.format(w_start, w_start + len(word) - 1))
                        words_chain.append(word)
                        words_chain.append(entity)
                      
                else:  # if by mistake a 'I-{*}' arises but there is no previous 'B-{*}'

                    words_chain.append(sId)
                    words_chain.append('{}-{}'.format(w_start, w_start + len(word) - 1))
                    words_chain.append(word)
                    words_chain.append(entity)
                    
        else:
            if len(words_chain) > 0:
                outfile.write("{}|{}|{}|{}\n".format(words_chain[0], words_chain[1], words_chain[2], words_chain[3]))
              
            words_chain = []
      
duration = time() - t0
print("done in %fs" % (duration))

Writing output TEST file
done in 0.132979s
